# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846890086845                   -0.70    4.8         
  2   -7.852323835819       -2.26       -1.53    1.0   25.0ms
  3   -7.852614568775       -3.54       -2.56    1.2   25.5ms
  4   -7.852646007980       -4.50       -2.90    2.8   34.2ms
  5   -7.852646526505       -6.29       -3.21    1.0   24.4ms
  6   -7.852646678906       -6.82       -3.99    1.0   24.6ms
  7   -7.852646686181       -8.14       -5.17    1.2   26.7ms
  8   -7.852646686726       -9.26       -5.57    2.8   34.6ms
  9   -7.852646686730      -11.40       -6.57    1.2   26.6ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846918467134                   -0.70           4.5         
  2   -7.852526948174       -2.25       -1.64   0.80    2.0    268ms
  3   -7.852636693971       -3.96       -2.72   0.80    1.0   24.3ms
  4   -7.852646416793       -5.01       -3.27   0.80    2.0   30.3ms
  5   -7.852646678410       -6.58       -4.06   0.80    1.5   27.2ms
  6   -7.852646686432       -8.10       -4.70   0.80    1.8   28.3ms
  7   -7.852646686716       -9.55       -5.72   0.80    1.5   27.2ms
  8   -7.852646686730      -10.87       -6.81   0.80    2.8   33.5ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.356008e+01     3.266151e+00
 * time: 0.0660090446472168
     1     1.272009e+00     1.957520e+00
 * time: 0.2987558841705322
     2    -1.447525e+00     2.260314e+00
 * time: 0.32352685928344727
     3    -3.792538e+00     1.803430e+00
 * time: 0.3586289882659912
     4    -5.260946e+00     1.625461e+00
 * time: 0.39374494552612305
     5    -6.814942e+00     9.814497e-01
 * time: 0.42914485931396484
     6    -6.996354e+00     1.163563e+00
 * time: 0.45377302169799805
     7    -7.523362e+00     7.027120e-01
 * time: 0.4787428379058838
     8    -7.612201e+00     6.877256e-01
 * time: 0.5036220550537109
     9    -7.750380e+00     2.124198e-01
 * time: 0.5285630226135254
    10    -7.789369e+00     1.133265e-01
 * time: 0.5533459186553955
    11    -7.817178e+00     1.388878e-01
 * time: 0.5780429840087891
    12    -7.835840e+00     8.337356e-02
 * time: 0.6023688316345215
    13    -7.847210e+00     2.911606e-02
 * time: 0.626646

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846867102438                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645923181                   -1.64         
  2   -7.852646686730       -6.12       -3.71    2.10s
  3   -7.852646686730      -13.35       -7.26    156ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.763049234080656e-7
|ρ_newton - ρ_scfv| = 1.7775788586954235e-7
|ρ_newton - ρ_dm|   = 1.5098492497461646e-9
